In [2]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt 
from torchvision.transforms import ToTensor, ToPILImage
import glob
import csv
import cv2 
import math
from shutil import copy
from skimage.metrics import structural_similarity as skssim
from skimage.metrics import peak_signal_noise_ratio as skpsnr
from skimage.measure import entropy, shannon_entropy

In [12]:
from grad import is_image_file

In [6]:
csv_path = "sobeltest.csv"
grad = {}
with open(csv_path, "r") as f:
    f_csv = csv.reader(f)
    #for i in f_csv:
    for k,v in f_csv:
        grad[k] = float(v)



In [14]:
def get_sorted_grad_list(csv_path):
    grad = {}
    with open(csv_path, "r") as f:
        f_csv = csv.reader(f)
        #for i in f_csv:
        for k,v in f_csv:
            grad[k] = float(v)
    sorted_grad = sorted(grad.items(),key=lambda x:x[1])
    grad_list = list(sorted_grad)
    return grad_list


In [7]:
sorted_grad_list = list(sorted(grad.items(),key=lambda x:x[1]))

In [8]:
sorted_grad_list[len(sorted_grad_list)//3]

('pond_42.jpg', 30.50060555555556)

In [9]:
sorted_grad_list[len(sorted_grad_list)//3*2]

('viaduct_48.jpg', 53.010172222222224)

In [10]:
csv_path = "sobeltrain.csv"
grad = {}
with open(csv_path, "r") as f:
    f_csv = csv.reader(f)
    #for i in f_csv:
    for k,v in f_csv:
        grad[k] = float(v)
sorted_grad_list = list(sorted(grad.items(),key=lambda x:x[1]))


In [11]:
sorted_grad_list[len(sorted_grad_list)//3]

('port_05.jpg', 33.12996944444444)

In [12]:
sorted_grad_list[len(sorted_grad_list)//3*2]

('industrial_21.jpg', 54.985800000000005)

In [86]:
os.mkdir("low_sobel_grad")
os.mkdir("middle_sobel_grad")
os.mkdir("high_sobel_grad")

In [46]:
for n,i in enumerate(sorted_grad_list):
    src = os.path.join(r"E:\Code\Python\datas\RS\WHU-RS19-test\GT",i[0])    
    if n < len(sorted_grad_list)//3:
        dst = os.path.join(r"low_sobel_grad",i[0])
    elif n < len(sorted_grad_list)//3*2:
        dst = os.path.join(r"middle_sobel_grad",i[0])
    else:
        dst = os.path.join(r"high_sobel_grad",i[0])
    copy(src,dst)


In [15]:
sorted_grad_list = get_sorted_grad_list("laplaciantrain.csv")

In [16]:
sorted_grad_list[len(sorted_grad_list)//3]

('river_39.jpg', 10.397541666666665)

In [17]:
sorted_grad_list[len(sorted_grad_list)//3*2]

('port_40.jpg', 18.999655555555556)

In [18]:
sorted_grad_list = get_sorted_grad_list("laplaciantest.csv")

In [19]:
sorted_grad_list[len(sorted_grad_list)//3]

('meadow_48.jpg', 8.833213888888888)

In [20]:
sorted_grad_list[len(sorted_grad_list)//3*2]

('railwayStation_43.jpg', 18.628952777777776)

In [22]:
# Sobel算子
def sobel_demo(image):
    grad_x = cv2.Sobel(image, cv2.CV_32F, 1, 0)  # x方向一阶导数
    grad_y = cv2.Sobel(image, cv2.CV_32F, 0, 1)  # y方向一阶导数
    gradx = cv2.convertScaleAbs(grad_x)  # 转回原来的uint8形式
    grady = cv2.convertScaleAbs(grad_y)
    gradxy = cv2.addWeighted(gradx, 0.5, grady, 0.5, 0)  # 图像融合
    return gradxy 

In [51]:
def class_grad(img_dir):
    all_grad = []
    for i in os.listdir(img_dir):
        path = os.path.join(img_dir, i)
        if not is_image_file(path):
            continue
        imgcv = cv2.imread(path)                
        imgcv = cv2.cvtColor(imgcv,cv2.COLOR_BGR2GRAY)
        h,w = imgcv.shape
        grad = sobel_demo(imgcv)
        mean_grad = np.sum(grad)/h/w
        print(f"{i} {mean_grad:.4f}")
        all_grad.append(mean_grad)
    mean = np.mean(all_grad)
    std = np.std(all_grad)
    return mean,std

In [44]:
whurs = r"E:\Code\Python\datas\RS\WHURS19"
class19 = os.listdir(whurs)
for c in class19:
    path =  os.path.join(whurs,c)
    mean, std = class_grad(path)
    print(f"{c:20} {mean:.4f}  {std:.4f}")

Airport              46.6438  9.8246
Beach                8.0693  5.5964
Bridge               20.7627  9.2148
Commercial           66.4293  10.5533
Desert               11.2931  4.3392
Farmland             20.5787  8.2156
footballField        41.9481  8.7321
Forest               52.7441  10.8066
Industrial           55.8971  9.4492
Meadow               19.1728  7.9876
Mountain             69.9471  18.9475
Park                 48.3128  12.8408
Parking              53.0995  14.6865
Pond                 26.9388  8.5575
Port                 42.9220  11.2007
railwayStation       63.5774  15.5679
Residential          66.0944  9.1859
River                46.2962  9.8576
Viaduct              62.9744  11.0396


In [43]:
whurs = r"E:\Code\Python\datas\RS\AID"
class30 = os.listdir(whurs)
for c in class30:
    path =  os.path.join(whurs,c)
    mean, std = class_grad(path)
    print(f"{c:20} {mean:.4f}  {std:.4f}")



Airport              36.5533  9.6303
BareLand             22.1218  11.5225
BaseballField        29.9471  7.0131
Beach                25.6375  16.9841
Bridge               28.3456  11.1251
Center               44.3729  14.3960
Church               57.7800  11.1239
Commercial           47.7580  13.0781
DenseResidential     61.9680  12.2234
Desert               13.3862  8.2887
Farmland             22.8848  11.1252
Forest               49.4652  17.5238
Industrial           50.0223  14.9248
Meadow               25.7716  10.8364
MediumResidential    48.1444  14.7813
Mountain             42.7011  16.6688
Park                 44.8740  12.2410
Parking              54.0840  12.4066
Playground           30.8634  10.1097
Pond                 29.4701  8.3792
Port                 38.7304  13.5667
RailwayStation       47.7738  15.0187
Resort               44.9158  15.1586
River                33.2066  10.8658
School               49.4786  13.3212
SparseResidential    51.3012  8.8502
Square           

In [53]:

mean, std = class_grad(r"E:\Code\Python\datas\SR\DIV2K\DIV2K_train_HR")
print(f"mean {mean:.4f}  std {std:.4f}")



0001.png 34.5643
0002.png 40.8545
0003.png 52.0210
0004.png 16.5929
0005.png 50.9632
0006.png 17.9743
0007.png 58.8278
0008.png 34.1645
0009.png 44.2190
0010.png 31.3905
0011.png 47.8717
0012.png 36.4642
0013.png 13.0826
0014.png 40.9217
0015.png 47.3862
0016.png 33.9530
0017.png 34.8840
0018.png 33.9635
0019.png 46.1197
0020.png 19.2761
0021.png 32.1177
0022.png 56.7908
0023.png 10.8037
0024.png 26.5807
0025.png 28.9869
0026.png 69.4361
0027.png 24.1743
0028.png 22.4304
0029.png 73.9399
0030.png 34.9159
0031.png 44.4733
0032.png 20.7306
0033.png 29.1283
0034.png 52.9856
0035.png 42.2598
0036.png 15.3587
0037.png 21.1594
0038.png 44.3838
0039.png 32.2539
0040.png 10.6530
0041.png 26.8838
0042.png 41.6734
0043.png 30.1039
0044.png 34.5819
0045.png 41.6905
0046.png 28.8367
0047.png 22.6795
0048.png 10.1261
0049.png 29.1290
0050.png 22.4406
0051.png 40.3717
0052.png 58.7806
0053.png 31.9489
0054.png 57.3550
0055.png 13.4821
0056.png 34.9228
0057.png 14.1335
0058.png 16.7890
0059.png 20.97